<a href="https://colab.research.google.com/github/glaiml/prj11-nlp-Project-1/blob/master/VGK_SeqNLP_Project1_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGK- IMDB-Sentiment Classification


## Loading the dataset

In [0]:
from keras.datasets import imdb

#vocab_size = 10000 #vocab size

# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

In [0]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

## Train test split

1. Import test and train data (5 Marks)
2. Import the labels ( train and test) (5 Marks)

In [0]:
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [19]:
print (x_train[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    1
  194 1153  194 8255   78  228    5    6 1463 4369 5012  134   26    4
  715    8  118 1634   14  394   20   13  119  954  189  102    5  207
  110 3103   21   14   69  188    8   30   23    7    4  249  126   93
    4  114    9 2300 1523    5  647    4  116    9   35 8163    4  229
    9  340 1322    4  118    9    4  130 4901   19    4 1002    5   89
   29  952   46   37    4  455    9   45   43   38 1543 1905  398    4
 1649 

In [20]:
print (x_train[1].shape)

(300,)


In [21]:
print(y_train[1])

0


In [22]:
import numpy as np

unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[    0     1]
 [12500 12500]]


In [23]:
unique_elements, counts_elements = np.unique(y_test, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[    0     1]
 [12500 12500]]


## WORD INDEX BUILDING

3. Get the word index and then Create a key-value pair for word and word_id (12.5 points)

In [0]:
word_index = imdb.get_word_index()

In [0]:
#ref :: https://stackoverflow.com/questions/41971587/how-to-convert-predicted-sequence-back-to-text-in-keras
reverse_word_map = dict(map(reversed, word_index.items()))

In [0]:
# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

In [27]:
#test
review = sequence_to_text(x_train[0])
print(review)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of'

## Build Keras Embedding Layer Model

4. Build a Sequential Model using Keras for the Sentiment Classification task (10 points)

We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [28]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import LSTM
### create the model
model = Sequential()
model.add(Embedding(vocab_size, 128, trainable=True, input_length=maxlen))
model.add(LSTM(units=64, dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
### Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=500, verbose=1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 128)          1280000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 1,331,521
Trainable params: 1,331,521
Non-trainable params: 0
_________________________________________________________________
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 44s 2ms/step - loss: 0.5664 - acc: 0.7094 - val_loss: 0.3631 - val_acc: 0.8521
Epoch 2/10
25000/25000 [==============================]

## Model Accuracy
5. Report the Accuracy of the model (5 points)

In [29]:
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.69%


In [0]:
y_pred = model.predict(x_test)

In [31]:
print(y_pred)

[[0.01475677]
 [0.9999519 ]
 [0.36325932]
 ...
 [0.00889832]
 [0.04148394]
 [0.89531636]]


In [0]:
y_pred = np.round(y_pred, 0)

In [33]:
y_pred = y_pred.ravel()
y_pred.shape

(25000,)

In [0]:
y_pred = y_pred.astype('int64')

In [35]:
y_test.ravel()
y_test

array([0, 1, 1, ..., 0, 0, 0])

In [36]:
from sklearn.metrics import classification_report
target_names = ['Sentiment_Positive', 'Sentiment_Negative']
print(classification_report(y_test, y_pred, target_names=target_names))

                    precision    recall  f1-score   support

Sentiment_Positive       0.85      0.87      0.86     12500
Sentiment_Negative       0.86      0.85      0.86     12500

          accuracy                           0.86     25000
         macro avg       0.86      0.86      0.86     25000
      weighted avg       0.86      0.86      0.86     25000



In [37]:
sequence_to_text(x_test[0])

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [38]:
sequence_to_text(x_test[1])

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'the',
 'as',
 'you',
 "world's",
 'is',
 'quite',
 'br',
 'mankind',
 'most',
 'that',
 'quest',
 'are',
 'chase',
 'to',
 'being',
 'quickly',
 'of',
 'little',
 'it',
 'time',
 'hell',
 'to',
 'plot',
 'br',
 'of',
 'something',
 'long',
 'put',
 'are',
 'of',
 'every',
 'place',
 'this',
 'consequence',
 'and',
 'of',
 'interplay',
 'storytelling',
 'being',
 'nasty',
 'not',
 'of',
 'you',
 'warren',
 'in',
 'is',
 'failed',
 'club',
 'i',
 'i',
 'of',
 'films',
 'pay',
 'so',
 'sequences',
 'and',
 'film',
 'okay',
 'uses',
 'to',
 'received',
 'and',
 'if',
 'time',
 'done',
 'for',
 'room',
 'sugar',
 'viewer',
 'as',
 'cartoon',
 'of',
 'gives',
 'to',
 'forgettable',
 'br',
 'be',
 'because',
 'many'

## Retrive the output of each layer in keras for a given single test sample from the trained model you built

6. Retrieve the output of each layer in Keras for a given single test sample from the trained model you built (2.5 points)

In [39]:
from keras import backend as K

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
print (outputs)
functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    # evaluation functions

# Testing
test = x_test[0][np.newaxis,...]
layer_outs = [func([test, 1.]) for func in functors]


print (layer_outs)

[<tf.Tensor 'embedding_2/embedding_lookup/Identity:0' shape=(?, 300, 128) dtype=float32>, <tf.Tensor 'lstm_2/TensorArrayReadV3:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'dense_3/Relu:0' shape=(?, 32) dtype=float32>, <tf.Tensor 'dense_4/Sigmoid:0' shape=(?, 1) dtype=float32>]
[[array([[[ 0.09708548, -0.04185277, -0.0233365 , ..., -0.04557361,
          0.04528384, -0.03951082],
        [ 0.09708548, -0.04185277, -0.0233365 , ..., -0.04557361,
          0.04528384, -0.03951082],
        [ 0.09708548, -0.04185277, -0.0233365 , ..., -0.04557361,
          0.04528384, -0.03951082],
        ...,
        [ 0.01653893,  0.00948247, -0.00811201, ..., -0.0069657 ,
         -0.00347264, -0.04094478],
        [-0.02197415,  0.00516459,  0.03317114, ..., -0.04325645,
         -0.02075824, -0.03856363],
        [-0.01060683, -0.02133838,  0.01332563, ...,  0.06967086,
         -0.04189531,  0.04644488]]], dtype=float32)], [array([[ 0.09414316, -0.00240031, -0.03494994, -0.02601207,  0.11273657,
  